1. Import modules and functions

In [2]:
import os
import sys
print(os.getcwd())
os.chdir("C:/Users/alexm/NUS Dropbox/Alexander Mozdzen/ocr/ocr_new") # change in the future
print(os.getcwd())
import glob
import pandas as pd
from pathlib import Path

from src.extract_and_create import *


C:\Users\alexm\NUS Dropbox\Alexander Mozdzen\ocr\ocr_new
C:\Users\alexm\NUS Dropbox\Alexander Mozdzen\ocr\ocr_new


ModuleNotFoundError: No module named 'src.extract_and_create'

2. Extract text regions and crop images

In [2]:

# def extract_and_create(input_folder, output_folder):
#     jpg_folder = input_folder / "jpg"
#     xml_folder = input_folder / "xml"

#     print(f"Processing images from: {jpg_folder}")
#     print(f"Processing XMLs from: {xml_folder}")

#     # List files in jpg and xml folders
#     #print("Files in jpg folder:")
#     #for file in jpg_folder.iterdir():
#     #    print(file)

#     #print("Files in xml folder:")
#     #for file in xml_folder.iterdir():
#     #    print(file)

#     image_paths = sorted(jpg_folder.glob("*.jpg"), key=lambda p: p.stem)
#     xml_paths = sorted(xml_folder.glob("*.xml"), key=lambda p: p.stem)

#     print(f"Found {len(image_paths)} images and {len(xml_paths)} XML files.")

#     for image_path, xml_path in zip(image_paths, xml_paths):
#         if image_path.stem == xml_path.stem:
#             base_name = image_path.stem
#             print(f"Processing: {base_name}")

#             regions = parse_xml_index_ordering(xml_path)
#             pd.DataFrame({
#                 "text": regions["ordered_text"],
#                 "identifier": [f"{base_name}_{region_id}" for region_id in regions["region_id"]]
#             }).to_csv(f"{output_folder}/texts/{base_name}.csv", index=False)

#             cropped_images = create_images_from_regions(image_path, regions,
#                                                                   buffer_above=0, buffer_below=0, buffer_left=0, buffer_right=0)

#             for cropped_image, region_id in cropped_images:
#                 cropped_image.save(f"{output_folder}/images/{base_name}_{region_id}.png")
#                 print(f"Saved image: {base_name}_{region_id}.png")


In [ ]:
input_folder = Path("data/raw/")
output_folder = Path("data/processed")
extract_and_create(input_folder, output_folder)

Now the part that was performed in 2_create